# Gensim Processing

In [4]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
import gensim
from collections import defaultdict
import contractions
import re
import string

In [5]:
eng_dict = set(words.words())
lemma = WordNetLemmatizer()
stops = set(stopwords.words('english'))

In [8]:
comments2 = pd.read_csv('../data/comments.csv', sep='|')
comments2.dropna(inplace=True)
text = list(comments2['text'])
strip_punctuation = re.compile('[%s]' % re.escape(string.punctuation))

In [9]:
# def expand_contractions(corpus):
#     expanded = [contractions.fix(comment.lower()) for comment in corpus]
#     return expanded

In [10]:
# def strip_punctuation(corpus):
#     strip_punc = re.compile('[%s]' % re.escape(string.punctuation))
#     no_punc = [strip_punc.sub('', comment) for comment in corpus]
#     return no_punc

In [44]:
class ProcessCorpus():
    '''
    Class for NLP processign to create a corpus that has:
      - contractions expanded
      - punctuation stripped
      - lemmatizaion
    There is an optional frequency filter included to remove words that appear less than some number of times.

    This class has stored attributes for a dictionary and corpus object to be supplied to gensim.
    '''

    def __init__(self, stopwords=None, eng_dict=None):
        self.stopwords = stopwords
        self.eng_dict = eng_dict
    
    def __expand_contractions(self):
        '''
        Expands contraction words such as don't --> do not prior to removing punctuation so meaning isn't lost and words will still appear in supplied dictionary for dictionary filtering.
        '''
        expanded = [contractions.fix(comment.lower()) for comment in self.corpus]
        return expanded

    def __strip_punctuation(self):
        '''
        Removes all punctuation from corpus.
        '''
        strip_punc = re.compile('[%s]' % re.escape(string.punctuation))
        no_punc = [strip_punc.sub('', comment) for comment in self.corpus]
        return no_punc

    def __lemmatize(self):
        '''
        Lemmatizes words using NLTK package's WordNetLemmatizer.
        '''
        split = []
        lemma = WordNetLemmatizer()
        for comment in self.corpus:
            split.append([lemma.lemmatize(word) for word in comment.split() if word not in self.stopwords])
        return split

    def __frequency_dict(self):
        '''
        Creates a dictionary of word frequencies in the corpus to be used for frequency filtering. Good for removing typos, and words that are not frequent enough to be considered slang due to low adoption by subculture.
        '''
        self.frequency_dict = defaultdict(int)
        for text in self.corpus:
            for token in text:
                self.frequency_dict[token] += 1

    def fit(self, corpus):
        '''
        Call to process corpus.

        Creates attributes:
        corpus - processed
        gensim_dictionary
        '''
        self.corpus = corpus
        self.corpus = self.__expand_contractions()
        self.corpus = self.__strip_punctuation()
        self.corpus = self.__lemmatize()
        self.__frequency_dict()
        self.gensim_dictionary = gensim.corpora.Dictionary(self.corpus)
        temp = self.gensim_dictionary[0]
        self.gensim_corpus = [self.gensim_dictionary.doc2bow(text) for text in self.corpus]

    def return_frequency_filter(self, frequency = 1):
        freq_filter = [
            [token for token in text if self.frequency_dict[token] > frequency]
            for text in self.corpus
        ]
        return freq_filter
    
    def apply_frequency_filter(self, frequency = 1):
        self.corpus_freq = self.return_frequency_filter(frequency)
        self.gensim_dictionary_freq = gensim.corpora.Dictionary(self.corpus)
        temp = self.gensim_dictionary_freq
        self.gensim_corpus_freq = [self.gensim_dictionary.doc2bow(text) for text in self.corpus]



In [23]:
dictionary = gensim.corpora.Dictionary(corpus.corpus)
c = [dictionary.doc2bow(text) for text in corpus.corpus]

In [36]:
corpus = ProcessCorpus(stopwords=stops, eng_dict=eng_dict)

In [37]:
corpus.fit(text)

In [48]:
corpus.apply_frequency_filter(5)

In [49]:
model = gensim.models.LdaModel(
    corpus=corpus.gensim_corpus,
    id2word=corpus.gensim_dictionary.id2token,
    chunksize=2000,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=10,
    passes=20,
    eval_every=1
)

In [46]:
corpus.gensim_dictionary_freq.id2token

{0: 'ama',
 1: 'asked',
 2: 'attending',
 3: 'better',
 4: 'duckworth',
 5: 'ended',
 6: 'event',
 7: 'everyone',
 8: 'fill',
 9: 'formhttpsdocsgooglecomformsyou2de1faipqlsdpb2ctl2lismo9xovs6sdqgdymyz29sewyqsntjy1oxpthwviewformuspsflink',
 10: 'future',
 11: 'going',
 12: 'lock',
 13: 'longterm',
 14: 'mod',
 15: 'question',
 16: 'see',
 17: 'senator',
 18: 'suggestion',
 19: 'thanks',
 20: 'thread',
 21: 'viewing',
 22: 'want',
 23: 'damaging',
 24: 'deconstructing',
 25: 'dejoy',
 26: 'get',
 27: 'go',
 28: 'hold',
 29: 'louis',
 30: 'need',
 31: 'postal',
 32: 'removed',
 33: 'service',
 34: 'stopped',
 35: 'agree',
 36: 'antagonistic',
 37: 'calling',
 38: 'change',
 39: 'changing',
 40: 'effective',
 41: 'either',
 42: 'example',
 43: 'feel',
 44: 'generally',
 45: 'give',
 46: 'like',
 47: 'made',
 48: 'mind',
 49: 'office',
 50: 'people',
 51: 'point',
 52: 'policy',
 53: 'pressure',
 54: 'public',
 55: 'reach',
 56: 'reaching',
 57: 'u',
 58: 'way',
 59: 'work',
 60: 'writing',

In [50]:
model.save('mdl/model2/full_corpus_model')

In [85]:
model.top_topics(corpus.gensim_corpus)

[([(0.02832856, 'like'),
   (0.026987018, 'I'),
   (0.025186889, 'people'),
   (0.02198579, 'would'),
   (0.016508155, 'get'),
   (0.016373083, 'one'),
   (0.012768851, 'think'),
   (0.01133419, 'know'),
   (0.011315112, 'make'),
   (0.0107678, 'time'),
   (0.010528573, 'thing'),
   (0.010364488, 'even'),
   (0.008684513, 'really'),
   (0.008629325, 'want'),
   (0.008394416, 'good'),
   (0.008318727, 'way'),
   (0.0082456, 'see'),
   (0.008190934, 'right'),
   (0.0081101265, 'look'),
   (0.007926063, 'go')],
  -2.215177324123479),
 ([(0.050953995, 'year'),
   (0.024527358, 'money'),
   (0.016512465, '2'),
   (0.01644555, 'last'),
   (0.015802735, 'month'),
   (0.014503769, 'job'),
   (0.013554154, '3'),
   (0.012370014, '1'),
   (0.011854476, 'high'),
   (0.011613844, 'school'),
   (0.011525462, 'ago'),
   (0.010895391, 'pay'),
   (0.010794938, 'free'),
   (0.010775934, 'buy'),
   (0.010216341, 'player'),
   (0.009171846, '100'),
   (0.00893493, '5'),
   (0.008806991, 'hour'),
   (0.00